In [1]:
import torch
import glob
import os
import json
from maml.metalearners import ModelAgnosticMetaLearning
from torch.utils.data import  DataLoader
from maml.datasets import get_benchmark_by_name
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

In [6]:

class Conv2dDataset(Dataset):
    def __init__(self):
        self.__save_path='/home/jaehun/workspace/pytorch-maml/GCN'
        self.__cost_path = os.path.join(self.__save_path,'conv2d','new_label.npy')
        self.__feature_path = os.path.join(self.__save_path,'conv2d','new_batch_1.npy')
        self.__costs=np.load(self.__cost_path).squeeze()
#         self.__costs=self.__costs[:,1:]
        self.__features=np.load(self.__feature_path).squeeze()
    def __len__(self):
        return len(self.__costs)
    def __getitem__(self, idx):
        return self.__features[idx],self.__costs[idx]

In [3]:
paths=glob.glob('results/graph_flops_order/**/config.json',recursive=True)
conv2d_dataset=Conv2dDataset()

In [4]:
tables=dict()
for path in paths:
    # 각 way or shot option 마다 ! 
    with open(path, 'r') as f:
        config = json.load(f)
        device = torch.device('cuda')
        benchmark = get_benchmark_by_name(config['dataset'],
                                          config['num_ways'],
                                          config['num_shots'],
                                          config['num_shots_test'],
                                          hidden_size=config['hidden_size'])
        metalearner = ModelAgnosticMetaLearning(benchmark.model,
                    first_order=config['first_order'],
                    num_adaptation_steps=config['num_steps'],
                    step_size=config['step_size'],
                    loss_function=benchmark.loss_function,
                    device=device)
        
        
        way_size=int(config['num_ways'])
        batch_size=way_size*2
        dataset_loader = DataLoader(conv2d_dataset,
                                                 batch_size=batch_size, shuffle=False,
                                                 num_workers=1)
        loss_before=[]
        loss_after=[]
        for idx,(data,label) in enumerate(dataset_loader):
            model=metalearner.model
            optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
            data=data.to(device)
            label=label.to(device)
            if data.shape[0]!=batch_size:
                break
            with open(config['model_path'], 'rb') as f:
                model.load_state_dict(torch.load(f, map_location=device))
            output=model(data[:way_size])
            loss=torch.nn.functional.l1_loss(output,label[:way_size])
            if idx %100==0:
                print(idx,'before loss adapt\n',float(loss))
                print('before\toutput\n',output.squeeze())
                print('before\tlabel\n',label[:way_size].squeeze())
                print()
            loss_before.append(float(loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            output=model(data[way_size:])
            loss=torch.nn.functional.l1_loss(output,label[way_size:])
            if idx %100==0:
                print(idx,'after loss adapt\n',float(loss))
                print('after\toutput\n',output.squeeze())
                print('after\tlabel\n',label[way_size:].squeeze())
                print()
            loss_after.append(float(loss))
            
            
        tables[f"num_ways_{config['num_ways']}_num_shots_{config['num_shots']}"]={'before':np.mean(loss_before),
                                                              'after':np.mean(loss_after)}
        
            
            
        




0 before loss adapt
 0.12874145805835724
before	output
 tensor([0.3424, 0.1257, 0.3793, 0.3424, 0.1481, 0.2744, 0.2354, 0.2190, 0.1205,
        0.3687, 0.3222, 0.1802, 0.2542, 0.2995, 0.3538, 0.2098, 0.1765, 0.3488,
        0.3371, 0.3190], device='cuda:0', grad_fn=<SqueezeBackward0>)
before	label
 tensor([0.1376, 0.1390, 0.1392, 0.1405, 0.1414, 0.1417, 0.1419, 0.1421, 0.1422,
        0.1434, 0.1436, 0.1438, 0.1441, 0.1445, 0.1448, 0.1458, 0.1460, 0.1468,
        0.1471, 0.1471], device='cuda:0')

0 after loss adapt
 0.08354370296001434
after	output
 tensor([0.2383, 0.2479, 0.1808, 0.2831, 0.2553, 0.2319, 0.0738, 0.0412, 0.0677,
        0.1847, 0.1816, 0.2358, 0.1852, 0.0696, 0.2286, 0.2732, 0.2643, 0.1269,
        0.3068, 0.2532], device='cuda:0', grad_fn=<SqueezeBackward0>)
after	label
 tensor([0.1472, 0.1476, 0.1478, 0.1479, 0.1482, 0.1483, 0.1486, 0.1488, 0.1493,
        0.1493, 0.1495, 0.1497, 0.1498, 0.1501, 0.1514, 0.1520, 0.1528, 0.1538,
        0.1544, 0.1548], device='cuda:0'

400 before loss adapt
 0.5162850022315979
before	output
 tensor([0.3796, 0.3653, 0.3567, 0.2916, 0.2821], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
before	label
 tensor([0.8511, 0.8511, 0.8515, 0.8515, 0.8516], device='cuda:0')

400 after loss adapt
 0.3531758785247803
after	output
 tensor([0.4474, 0.5001, 0.4593, 0.5216, 0.5658], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
after	label
 tensor([0.8517, 0.8519, 0.8520, 0.8522, 0.8522], device='cuda:0')

0 before loss adapt
 0.25266873836517334
before	output
 tensor([0.4213, 0.3290, 0.4668, 0.4213, 0.3227], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
before	label
 tensor([0.1376, 0.1390, 0.1392, 0.1405, 0.1414], device='cuda:0')

0 after loss adapt
 0.12059785425662994
after	output
 tensor([0.2096, 0.3147, 0.2708, 0.1802, 0.3389], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
after	label
 tensor([0.1417, 0.1419, 0.1421, 0.1422, 0.1434], device='cuda:0')

100 before loss adapt
 0.12891358137130737
before	ou

In [5]:
df=pd.DataFrame(tables)
df.to_csv('graph_flops_order_MAE.csv',index=False)

In [7]:
model

MetaMLPModel(
  (features): MetaSequential(
    (layer1): MetaSequential(
      (linear): MetaLinear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
    )
    (layer2): MetaSequential(
      (linear): MetaLinear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
    )
    (layer3): MetaSequential(
      (linear): MetaLinear(in_features=128, out_features=64, bias=True)
      (relu): ReLU()
    )
  )
  (classifier): MetaLinear(in_features=64, out_features=1, bias=True)
)